<center><h2>Tutorial 1 - Getting Started</h2>
<br>Mary K. Coe <br>
5 Jan 2022</center>

This is the first in a series of four tutorials which will guide you through how to use this Monte Carlo program. This program was written as part of my PhD and allows simulation of a truncated Lennard-Jones fluid or Molinero and Moore's monatomic water in bulk, confined to a slit or in contact with a solute. The Monte Carlo program is written in C++ and is accompanied by Python scripts which run and analyse the results.

The purpose of this tutorial is to help you setup the program and ensure it is working correctly. By the end of the tutorial, you will have

    - Added a random number generator into the program.
    - Tested the random number generator is working correctly.
    
If you are intersted in where this program has been used, you can check out the following papers:

M. K. Coe, R. Evans and N. B. Wilding, Density depletion and enhanced fluctuations in water near hydrophobic solutes: Identifying the underlying physics. <i>Phys. Rev, Lett.</i> (Accepted/In Press)

as well as my thesis, described below.

<h3>Introduction</h3>
These tutorials assume that you are already familiar with the theory behind Monte Carlo and its use in the simulation of liquids. If this is not the case, you may wish to consult the following references before continuing:

D. Frenkel and B. Smit, <i>Understanding Molecular Simulation, From Algorithms to Applications</i>, 2nd ed. (Academic Press, London, 2002). - This book is an excellent resource for understanding both Molecular Dynamics and Monte Carlo, and covers both the basics as well as advanced topics.

M. P. Allen and D. J. Tildesley, <i>Computer Simulation of Liquids</i>, 2nd ed. (Oxford University Press, Oxford, 2017). - This book covers details on the theory and implementation of the simulation of liquids. It is an excellent standard resource.

<a href = https://research-information.bris.ac.uk/ws/portalfiles/portal/304220732/Thesis_Mary_Coe.pdf>M. K. Coe, "Hydrophobicity across length scales: The role of surface criticality", PhD thesis, (University of Bristol, 2021)</a> - My thesis. Section 4.2 covers the Monte Carlo theories and methods used within the program in depth.

The first two of these references will give you a good foundation, whilst the latter goes into detail about the exact methods used within this particular program. As you progress through these tutorials more advanced techniques will be introduced. When this happens, references will be given accordingly.

In addition to an understanding of Monte Carlo and simulation of liquids, you will need an installation of C++ 2017 with the standard libraries as well as the g++ compiler. You will also need an installation of Python with the standard libraries. Throughout these tutorials it will be assumed that you have knowledge of Python. Knowledge of C++ is less essential, though you will need to add a random number generator to the program. Detailed steps for adding specific generators will be given, and those unfamiliar with C++ are recommended to follow these.

<h4>Program Layout</h4>
As mentioned, the Monte Carlo program itself, the source files for which are found within the <i>Program/src</i> folder, is written in C++ and is compiled using a Makefile. However, for various reasons (most notably ease of use on a cluster), I wrote several scripts which automate the running and analysis of the program in Python. These are stored within the <i>tools</i> folder. Throughout these tutorials, it is rare that you will interface directly with the Monte Carlo program. Instead, these tutorials will typically call Python scripts from the <i>tools</i> folder, which in turn will call the Monte Carlo program. 

<b>Note:</b> The Makefile written for the program is designed for Linux (and Mac) operating systems. If using Windows, you will need to edit it accordingly. This process is not described in the tutorials. If using Windows, it is recommended you compile the C++ program on Linux and then copy the executable to your Windows machine. 

<h3>Adding a Random Number Generator</h3>

The Monte Carlo program requires a random number generator in order to work. Such a generator is not included in the source code, and it is therefore up to the user to add one. Whilst you are free to choose any generator you like, it is important to note that your choice of generator may affect your results. There is a long history of research and debate over the best random number generators to use for Monte Carlo. 

Within this section we will first outline how to include a random number generator generally. This will be followed by detailed instructions on how to include two specific random number generators - ranvec by Prof B. Duenweg and Mersenne Twister. These latter two are just examples of generators you may choose. It is up to you to decide what random number generator is best for your research needs. 

<h4>Adding a General Random Number Generator</h4>

The generation of random numbers within the program is handled within the <i>'simbox.cpp'</i> file. This, and the associated header file, <i>'simbox.h'</i>, are the only files within the C++ program you will need to change in order to add your random number generator. To add the generator:

<b>Step 1:</b> First you will need to include the generator. You should do this by including the library or file containing the generator in the <i>include</i> statements at the top of the header file, <i>simbox.h</i>.

<b>Step 2:</b> You then need to initialise the generator and seed it. This should be done within the indicated space of the <i>setup</i> function in <i>simbox.cpp</i>. <b>Note the program is designed to use the computer time as the seed, if a seed has not been specificed in the Input file (which will be discussed later)</b>. Using the computer time ensures a unique seed for each simulation however the user may wish to change this.

<b>Step 3:</b> After initialising and seeding the generator, you must edit the <i>get_rnd</i> function within <i>simbox.cpp</i> such that it returns a random number from the generator when called. 

<b>Step 4:</b> Finally, if your random number generator is not a standard library, you will need to add a linking statement (and compiliation statement if applicable) to the Makefile. To add a compilation statement, follow the form of other compilation statements within the Makefile. Then add to the object file name (which should be the filename with the extension replaced with .o) to the end of the 'OBJECTS = ' line, leaving a space before. Finally, to link the random number generator, add the header file of the generator to the line starting <i>'\$(OBJDIR)simbox.o:'</i> following the same form as other linked object files in the line.

You should then compile the program by typing <i>'make'</i> into a terminal (note you should be in the same directory as the folders 'Program' and 'tools'), to check for compile errors. If any are found, you will need to debug the code you have added.

<b>Note:</b> Depending on how your random generator works, you may need to include additional functions. For example, some random number generators output an array of random numbers rather than give a single number each time. In this case, <i>get_rnd</i> would cycle through the numbers, continuously checking whether you'd been through all the numbers yet. If you had, then it would need to generate new numbers. There are some empty functions and variables you can use within the program if this is the case - see the <i>check_rnd</i> function and the variables <i>rand_check</i> and <i>random_numbers</i>, the latter of which are defined in the class definition in <i>simbox.h</i>.

<h4>A Specific Generator - ranvec.c</h4>
Early on in my PhD the random number generator recommended to me by my supervisor was <i>ranvec.c</i> by Prof B. Duenweg, which can be found <a href=https://www2.mpip-mainz.mpg.de/~duenweg/Public/PDFsOfPreprints/ranvec.c>here</a>. This random number generator is of the latter kind discussed above - it generates a large number of random numbers at a time (set to 10000 within the program by the nrands variable in <i>contants.h</i>). It is also important to note that this is a C program - rather than a C++ program - hence the include statement and compilation statement within the Makefile vary from those expected within C++. You will also need the C compiler <i>gcc</i>.

To use <i>ranvec.c</i>, follow these steps:

<b>Step 1:</b> You should download<a href=https://www2.mpip-mainz.mpg.de/~duenweg/Public/PDFsOfPreprints/ranvec.c> <i>ranvec.c</i></a> and add the file to the source code in the <i>Program/src</i> folder.

<b>Step 2:</b> Next you need to create a header file for <i>ranvec.c</i>. This will prevent any linking issues during compilation. To do this, create a new file within the <i>Program/src</i> folder and save it as <i>ranvec.h</i>. Then, copy the cody below into the file and save it.
    
    #ifndef RANVEC_H
    #define RANVEC_H

    void init_vector_random_generator(int iseed, int nrand);
    void vector_random_generator(int nrand, double *random_numbers);
    void write_random_generator(void);
    void read_random_generator(void);

    #endif
    
Note you may also wish to move the <i>#include</i> statements from <i>ranvec.c</i> into this file. If you do this, put them between the header guard but before the function declarations (below <i>#define RANVEC_H</i> but before <i>void init_vector_random_generator...</i>). If you do this, you will also need to add the statement <i>#include "ranvec.h"</i> where the include statements previously existed in <i>ranvec.c</i>.

<b>Step 3:</b> You then need to add an include statement into the <i>simbox.h</i> file. After the line <i>#include solute.h</i>, paste the following code:

    extern "C"{
    #include "ranvec.h"
    }

<b>Step 4:</b> The next step is to seed the generator. Paste the lines:
    
    init_vector_random_generator(seed, nrands);
    vector_random_generator(nrands, random_numbers);
    
into the indicated space within the <i>setup</i> function of <i>simbox.cpp</i>. 

<b>Step 5:</b> You will now tell the program how to get a random number. Paste the lines:

    rnd = random_numbers[rand_check];
    rand_check = rand_check+1;
    check_rnd();
    
into the indicated space within the <i>get_rnd</i> function within the <i>simbox.cpp</i> file. Then, paste the line:

    vector_random_generator(nrands, random_numbers);

into the indicated space of the <i>check_rnd</i> function just above. After doing this, you will have included the random number generator into the program.

<b>Step 6:</b> The final step is to edit the <i>Makefile</i>, to tell the program how to compile and link. Open the Makefile, and add the lines:

    $(OBJDIR)ranvec.o: $(SRCDIR)ranvec.c $(SRCDIR)ranvec.h
            gcc -O3 -c $< -o $@
   
just above the line beginning '<i>\$(OBJDIR)particle.o'</i>. <b>Note that the indentation of the second line should be tabs not spaces. Please check this before trying to compile!</b>. Next, add the statement <i>\$(SRCDIR)ranvec.h</i> to the end of the line beginning <i>'\$(OBJDIR)simbox.o:'</i>, leaving a space before. Finally, add the statement <i>$(OBJDIR)ranvec.o</i> to the end of the line beginning <i>OBJECTS=</i> at the top of the file, leaving a space before. Save the Makefile.
    
You should now check the program compiles by typing <i>make</i> into a terminal within the directory containing the <i>Makefile</i>.

<h4>A Specific Generator - Mersenne Twister</h4>
Mersenne twister is a well established and widely used random number generator which appears within standard C++ libraries. The latter feature means that it is very easy to use and to add into the program. 

To add Mersenne Twister:

<b>Step 1:</b> First, open the <i>simbox.h</i> file within the <i>Program/src</i> folder. Below the last <i>#include</i> statement, add the line:
    
    #include <random>

<b>Step 2:</b> After the last variable declaration (around line 65) within the private variables of the <i>sim_box</i> class declaration in <i>simbox.h</i>, add the lines:

     std::uniform_real_distribution<double> uniform_dist;
	 std::mt19937 rng;

<b>Step 3:</b> Open the file <i>simbox.cpp</i> in the <i>Program/src</i> folder. One of the first lines after the comment at the top of the file should read 

    sim_box::sim_box() {

Change this line to

    sim_box::sim_box(): uniform_dist(0.0,1.0) {

<b>Step 4:</b> Next, look for the <i>setup</i> function within the same file. Where indicated, add the line:

    rng.seed(seed);

to seed the random number generator.

<b>Step 5:</b> Scroll down within the same file until you reach the <i>get_rnd</i> function. Where indicated, add the line:
    
    rnd = uniform_dist(rng);
    
Save all the open files, and then check that the program compiles by typing <i>make</i> into a terminal. 
    

<h3>Testing</h3>
Once you have successfully included your random number generator of choice, you should test that the program is working as expected. Two tests have been designed to do this, and can be accessed using the <i>test.py</i> module within the <i>tools</i> folder. We will introduce each of these tests in turn, however first we must import the module.

In [1]:
import tools.test as test

<h4>Cell List Test</h4>

The first test ensures that the cell list - used within the program to keep track of particles - is setup correctly. If you do not know what a cell list is, or why it is useful, you should consult Appendix F.2 of <i>Understanding Molecular Simulation by Daan Frenkel and Berend Smit</i> (see Introduction for a full reference). Details of how cell lists are used within this program specifically can also be found in Section 4.2.3 of <a href =https://research-information.bris.ac.uk/ws/portalfiles/portal/304220732/Thesis_Mary_Coe.pdf>my thesis</a>. 

This test works by having the Monte Carlo program setup the cell list and then output it. This is then compared to a cell list generated by the Python tools function. If the two match exactly, then the test is passed. If not, then the test is failed. If the test fails, please download a fresh copy of the program and add your random number generator from scratch. 

In [2]:
test.cell_list()

*******************************************************************************
Grand Canonical Monte Carlo Program for simulating truncated Lennard-Jones
or monatomic water liquid either in bulk, confined to a slit, or in contact
with a solute.

Copyright (c) Mary Coe (2022)
Made available under the MIT License.
*******************************************************************************

Running cell list test...

----------------------
CELL LIST TEST PASSED
----------------------


<h4>Metropolis Test</h4>

This test specifically checks that the random number generator and acceptance criteria within the Monte Carlo program are working as expected and most importantly - obeying physics. According to physics, for an ideal gas $\mu/k_{\mathrm{B}}T \sim \ln<\rho>$, where $\mu$ is the chemical potential, $k_{\mathrm{B}}$ is Boltzmann's constant, $T$ is the temperature and $<\rho>$ is the average density. A good way to ensure the acceptance criteria is working is therefore to test whether this relation holds. This can be done by calling test function <i>metropolis</i>, named after the acceptance criteria used within the program.

In [3]:
test.metropolis()

*******************************************************************************
Grand Canonical Monte Carlo Program for simulating truncated Lennard-Jones
or monatomic water liquid either in bulk, confined to a slit, or in contact
with a solute.

Copyright (c) Mary Coe (2022)
Made available under the MIT License.
*******************************************************************************

Running metropolis test...
This will take a few minutes.
Simulation Complete.

Move Acceptance: 2388.38951%
Average Density: 0.04975
Average Energy: 0.00000
Running and End energy agreed to a tolerance of 0.00000

-----------------------------------------
Metropolis Test Results
Chemical Potential = -3.00000
ln(<density>) = -3.00072
Agreement = 0.00072
Relative Error = 0.00024
-----------------------------------------
TEST PASSED
-----------------------------------------


If this test is not obeyed, there is likely a problem with your random number generator. Go back and check that it is included and working as you expect. Keep doing this until the Metropolis test is passed.

<h3>Up Next</h3>
If both the Metropolis and Cell List tests are passed then you are ready to move onto tutorial 2. Tutorial 2 will look at the two model fluids available within the program, and how to run a bulk simulation of each. Particular attention will be paid to coexistence simulations and to finding the critical point. 